In [10]:
# 1. Setup
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from Models.NGRAM import NGramModel, classify

# 2. Load Dataset
df = pd.read_csv('AI_HUMAN.csv')
print(f"Total examples: {len(df)}")
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Total examples: 487235


,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [11]:
# 3. Train/Val Split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train size: {len(train_df)}, Val size: {len(val_df)}")

human_train = train_df[train_df['generated'] == 0]['text']
ai_train = train_df[train_df['generated'] == 1]['text']

Train size: 389788, Val size: 97447


In [13]:
nltk.download('punkt_tab')

# 4. Initialize and Train Models
n = 3
k = 0.5

model_human = NGramModel(n=n, k=k)
model_ai = NGramModel(n=n, k=k)

model_human.train(human_train)
model_ai.train(ai_train)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [14]:
# 5. Predict and Evaluate
val_texts = val_df['text']
val_labels = val_df['generated']

val_preds = val_texts.apply(lambda x: classify(model_human, model_ai, x))

acc = accuracy_score(val_labels, val_preds)
prec = precision_score(val_labels, val_preds)
rec = recall_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds)

print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

Accuracy:  0.9941
Precision: 1.0000
Recall:    0.9842
F1 Score:  0.9920
